In [3]:
from selenium.common.exceptions import NoSuchElementException, ElementClickInterceptedException
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
import time
import pandas as pd

In [24]:
def get_jobs(keyword, location, num_jobs, verbose, path, slp_time):
#def get_jobs(keyword, num_jobs, verbose, path, slp_time):
    
    '''Gathers jobs as a dataframe, scraped from Glassdoor'''
    
    #Initializing the webdriver
    options = webdriver.ChromeOptions()
    
    #Uncomment the line below if you'd like to scrape without a new Chrome window every time.
    #options.add_argument('headless')
    
    #Change the path to where chromedriver is in your home folder.
    driver = webdriver.Chrome(executable_path=path, options=options)
    driver.set_window_size(1120, 1000)


    driver.get('https://www.glassdoor.com/Job/jobs.htm?suggestCount=0&suggestChosen=false&clickSource=searchBtn&typedKeyword=&sc.keyword=&locT=&locId=&jobType=')
    
    #Uses first variable in function to input job title
    search = driver.find_element_by_id("KeywordSearch")
    search.send_keys(keyword)
    
    #Uses second variable in function to input Location.  Use 'City,State Abbreviation'
    search = driver.find_element_by_id("LocationSearch").clear()
    search = driver.find_element_by_id("LocationSearch")
    search.send_keys(location)
    search.send_keys(Keys.RETURN)
    
    #Test for the "Sign Up" prompt and get rid of it.
    time.sleep(slp_time)
    
    try:
                driver.find_element_by_class_name("selected").click()
    except ElementClickInterceptedException:
        pass
    
    time.sleep(.1)
    
    try:
            driver.find_element_by_css_selector('[alt="Close"]').click()  #clicking to the X.
    except NoSuchElementException:
        pass
    
    #changes 'posted' dropdown to 'Last Week'
    driver.find_element_by_id('filter_fromAge').click()
    time.sleep(5)
    driver.find_element_by_xpath('.//ul[@class="css-1dv4b0s ew8xong0"]//li[@value="7"]').click()
   
    
    
    jobs = []

    while len(jobs) < num_jobs:  #If true, should be still looking for new jobs.

        #Let the page load. Change this number based on your internet speed.
       
        time.sleep(5)
        
        
        #Going through each job in this page
        job_buttons = driver.find_elements_by_class_name("jl")  #jl for Job Listing. These are the buttons we're going to click.
        for job_button in job_buttons:  

            print("Progress: {}".format("" + str(len(jobs)) + "/" + str(num_jobs)))
            if len(jobs) >= num_jobs:
                break

            driver.execute_script("arguments[0].click();", job_button)  #You might 
            time.sleep(1)
            collected_successfully = False
            
            while not collected_successfully:
                try:
                    company_name = driver.find_element_by_xpath('.//div[@class="employerName"]').text
                    location = driver.find_element_by_xpath('.//div[@class="location"]').text
                    job_title = driver.find_element_by_xpath('.//div[contains(@class, "title")]').text
                    job_description = driver.find_element_by_xpath('.//div[@class="jobDescriptionContent desc"]').text
                    collected_successfully = True
                except:
                    time.sleep(5)

            try:
                salary_estimate = driver.find_element_by_xpath('.//div[@class = "salary"]').text

            except NoSuchElementException:
                salary_estimate = -1 #You need to set a "not found value. It's important."
            
            try:
                rating = driver.find_element_by_xpath('.//span[@class="rating"]').text
            except NoSuchElementException:
                rating = -1 #You need to set a "not found value. It's important."

            #Printing for debugging
            if verbose:
                print("Job Title: {}".format(job_title))
                print("Salary Estimate: {}".format(salary_estimate))
                print("Job Description: {}".format(job_description[:500]))
                print("Rating: {}".format(rating))
                print("Company Name: {}".format(company_name))
                print("Location: {}".format(location))

            #Going to the Company tab...
            #clicking on this:
            #<div class="tab" data-tab-type="overview"><span>Company</span></div>
            try:
                driver.find_element_by_xpath('.//div[@class="tab" and @data-tab-type="overview"]').click()

                try:
                    size = driver.find_element_by_xpath('.//div[@class="infoEntity"]//label[text()="Size"]//following-sibling::*').text
                except NoSuchElementException:
                    size = -1

                try:
                    founded = driver.find_element_by_xpath('.//div[@class="infoEntity"]//label[text()="Founded"]//following-sibling::*').text
                except NoSuchElementException:
                    founded = -1

                try:
                    type_of_ownership = driver.find_element_by_xpath('.//div[@class="infoEntity"]//label[text()="Type"]//following-sibling::*').text
                except NoSuchElementException:
                    type_of_ownership = -1

                try:
                    industry = driver.find_element_by_xpath('.//div[@class="infoEntity"]//label[text()="Industry"]//following-sibling::*').text
                except NoSuchElementException:
                    industry = -1

                try:
                    sector = driver.find_element_by_xpath('.//div[@class="infoEntity"]//label[text()="Sector"]//following-sibling::*').text
                except NoSuchElementException:
                    sector = -1

                try:
                    revenue = driver.find_element_by_xpath('.//div[@class="infoEntity"]//label[text()="Revenue"]//following-sibling::*').text
                except NoSuchElementException:
                    revenue = -1


            except NoSuchElementException:  #Rarely, some job postings do not have the "Company" tab.
                size = -1
                founded = -1
                type_of_ownership = -1
                industry = -1
                sector = -1
                revenue = -1
               

                
            if verbose:
                print("Size: {}".format(size))
                print("Founded: {}".format(founded))
                print("Type of Ownership: {}".format(type_of_ownership))
                print("Industry: {}".format(industry))
                print("Sector: {}".format(sector))
                print("Revenue: {}".format(revenue))
                print("@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@")

            jobs.append({"Job Title" : job_title,
            "Salary Estimate" : salary_estimate,
            "Job Description" : job_description,
            "Rating" : rating,
            "Company Name" : company_name,
            "Location" : location,
            "Size" : size,
            "Founded" : founded,
            "Type of ownership" : type_of_ownership,
            "Industry" : industry,
            "Sector" : sector,
            "Revenue" : revenue,})
            #add job to jobs

        #Clicking on the "next page" button
        try:
            driver.find_element_by_xpath('.//li[@class="next"]//a').click()
        except NoSuchElementException:
            print("Scraping terminated before reaching target number of jobs. Needed {}, got {}.".format(num_jobs, len(jobs)))
            break

    return pd.DataFrame(jobs)  #This line converts the dictionary object into a pandas DataFrame.

In [28]:
# path = "C:/Users/dell/Documents/R and Python/Glassdoor job/chromedriver"

# df = gs.get_jobs('data scientist', 'Columbus, OH', 39, False, path, 15)

# df.to_csv('C:/Users/Jen butt/Desktop/R and Python/Glassdoor job/DS-job-decision-tree/Jobs_list.csv')

path = "C:/Users/dell/Documents/ds_salary_proj/chromedriver"

df = get_jobs('data scientist', 'United States', 50, False, path, 15)
#df = get_jobs('data scientist', 70, False, path, 15)

Progress: 0/100
Progress: 1/100
Progress: 2/100
Progress: 3/100
Progress: 4/100
Progress: 5/100
Progress: 6/100
Progress: 7/100
Progress: 8/100
Progress: 9/100
Progress: 10/100
Progress: 11/100
Progress: 12/100
Progress: 13/100
Progress: 14/100
Progress: 15/100
Progress: 16/100
Progress: 17/100
Progress: 18/100
Progress: 19/100
Progress: 20/100
Progress: 21/100
Progress: 22/100
Progress: 23/100
Progress: 24/100
Progress: 25/100
Progress: 26/100
Progress: 27/100
Progress: 28/100
Progress: 29/100
Progress: 30/100
Progress: 31/100
Progress: 32/100
Progress: 33/100
Progress: 34/100
Progress: 35/100
Progress: 36/100
Progress: 37/100
Progress: 38/100
Progress: 39/100
Progress: 40/100
Progress: 41/100
Progress: 42/100
Progress: 43/100
Progress: 44/100
Progress: 45/100
Progress: 46/100
Progress: 47/100
Progress: 48/100
Progress: 49/100
Progress: 50/100
Progress: 51/100
Progress: 52/100
Progress: 53/100
Progress: 54/100
Progress: 55/100
Progress: 56/100


StaleElementReferenceException: Message: stale element reference: element is not attached to the page document
  (Session info: chrome=87.0.4280.141)


In [12]:
df.to_csv('glassdoor_jobs.csv', index = False)

In [13]:
df

,Job Title,Salary Estimate,Job Description,Rating,Company Name,Location,Size,Founded,Type of ownership,Industry,Sector,Revenue
0,Data Engineer- Hadoop,-1,Good Day !\n\nZensar Technologies is a leading...,3.4,Zensar Technologies\n3.4,Johannesburg,5001 to 10000 Employees,2001,Company - Public,IT Services,Information Technology,$100 to $500 million (USD)
1,Senior Analytic Consultant,-1,Why this role is critical to us\n\nThe Senior ...,4.1,Experian\n4.1,Bryanston,10000+ Employees,1980,Company - Public,Financial Analytics & Research,Finance,$2 to $5 billion (USD)
2,Data Scientist,-1,Purpose of Role\nOffering the opportunity to h...,4.0,FXCM\n4.0,Sandton,501 to 1000 Employees,1999,Company - Private,Investment Banking & Asset Management,Finance,$100 to $500 million (USD)
3,Data Scientist,-1,"Founded in South Africa in 2007, Praekelt.org ...",3.8,Praekelt.Org\n3.8,Johannesburg,51 to 200 Employees,-1,Nonprofit Organization,-1,-1,Less than $1 million (USD)
4,Data Scientist,-1,A well-known Insurance company in Johannesburg...,3.2,Hire Resolve\n3.2,Johannesburg,1 to 50 Employees,-1,Company - Private,Staffing & Outsourcing,Business Services,$500 million to $1 billion (USD)
5,Data Scientist,-1,KPIs:\nData Science\nContinuous improvement\nD...,-1,ClarkHouse,Johannesburg,1 to 50 Employees,2011,Company - Private,Consulting,Business Services,Less than $1 million (USD)
6,Data Scientist,-1,Udacity's mission is to train the world’s work...,3.6,Udacity\n3.6,Remote,201 to 500 Employees,2011,Company - Private,Education Training Services,Education,Unknown / Non-Applicable
7,Data Scientist,-1,"Information Technology, Gauteng JHB - Northern...",4.0,Boardroom Appointments\n4.0,Johannesburg,51 to 200 Employees,-1,Company - Private,-1,-1,Less than $1 million (USD)
8,Data Scientist,-1,11 Dec '20\nData Scientist\nInformation Techno...,2.4,boardroom\n2.4,Johannesburg,201 to 500 Employees,1968,Company - Public,Consulting,Business Services,$50 to $100 million (USD)
9,Senior Data Scientist,-1,"South Africa, Gauteng\n70000 to 135000 ZAR\nOt...",3.5,Jefferson Frank\n3.5,Johannesburg,1001 to 5000 Employees,2006,Company - Private,Staffing & Outsourcing,Business Services,Unknown / Non-Applicable
